In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model

In [2]:
train_dir = "../data/raf-db/train"
test_dir = "../data/raf-db/test"

In [3]:
train_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True
)

test_gen = ImageDataGenerator(rescale=1./255)

train_data = train_gen.flow_from_directory(
    train_dir,
    target_size=(224,224),
    batch_size=32,
    class_mode='categorical'
)

test_data = test_gen.flow_from_directory(
    test_dir,
    target_size=(224,224),
    batch_size=32,
    class_mode='categorical'
)

Found 12271 images belonging to 7 classes.
Found 3068 images belonging to 7 classes.


In [4]:
base_model = MobileNetV2(
    input_shape=(224,224,3),
    include_top=False,
    weights='imagenet'
)

for layer in base_model.layers:
    layer.trainable = False

In [5]:
x = base_model.output
x = Flatten()(x)
x = Dropout(0.5)(x)
predictions = Dense(7, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

In [6]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [7]:
history = model.fit(
    train_data,
    validation_data=test_data,
    epochs=10
)

Epoch 1/10
384/384 ━━━━━━━━━━━━━━━━━━━━ 362s 922ms/step - accuracy: 0.4655 - loss: 6.3364 - val_accuracy: 0.5385 - val_loss: 5.2487
Epoch 2/10
384/384 ━━━━━━━━━━━━━━━━━━━━ 361s 939ms/step - accuracy: 0.5304 - loss: 6.9542 - val_accuracy: 0.5365 - val_loss: 6.5705
Epoch 3/10
384/384 ━━━━━━━━━━━━━━━━━━━━ 355s 925ms/step - accuracy: 0.5540 - loss: 7.6107 - val_accuracy: 0.5968 - val_loss: 6.2597
Epoch 4/10
384/384 ━━━━━━━━━━━━━━━━━━━━ 402s 1s/step - accuracy: 0.5610 - loss: 8.0145 - val_accuracy: 0.5949 - val_loss: 6.6810
Epoch 5/10
384/384 ━━━━━━━━━━━━━━━━━━━━ 514s 1s/step - accuracy: 0.5700 - loss: 8.4479 - val_accuracy: 0.5596 - val_loss: 7.4973
Epoch 6/10
384/384 ━━━━━━━━━━━━━━━━━━━━ 515s 1s/step - accuracy: 0.5805 - loss: 8.8136 - val_accuracy: 0.5883 - val_loss: 7.1810
Epoch 7/10
384/384 ━━━━━━━━━━━━━━━━━━━━ 569s 1s/step - accuracy: 0.5821 - loss: 8.8765 - val_accuracy: 0.5844 - val_loss: 8.5421
Epoch 8/10
384/384 ━━━━━━━━━━━━━━━━━━━━ 522s 1s/step - accuracy: 0.5942 - loss: 9.2439 -

In [8]:
model.save("../models/emotion_model.h5")